<a href="https://colab.research.google.com/github/sau-coder/NLP/blob/master/Spam_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/spam.csv')

In [ ]:
df

In [ ]:
X = list(df["Messages"])
Y = list(df["label"])

In [ ]:
Y = list(pd.get_dummies(Y , drop_first = True )["spam"])

In [ ]:
from sklearn.model_selection import train_test_split
x_train , x_test , y_train , y_test = train_test_split(X , Y , random_state = 0)

In [ ]:
pip install transformers

In [ ]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [ ]:
train_encodings = tokenizer(x_train , truncation = True , padding = True)
test_encodings = tokenizer(x_test , truncation = True , padding = True) 

In [ ]:
pip install tensorflow

In [ ]:
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings) , y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings) , y_test))

In [ ]:
from transformers import TFDistilBertForSequenceClassification , TFTrainer , TFTrainingArguments

training_args = TFTrainingArguments(
    output_dir = './results',
    num_train_epochs = 2,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 16,
    eval_steps = 10,
    warmup_steps  = 500,
    weight_decay = 0.01,
    logging_dir = './logs',
    logging_steps = 10,
)

In [ ]:
with training_args.strategy.scope():
  model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

  trainer = TFTrainer(
      model = model,
      args = training_args,
      train_dataset = train_dataset,
      eval_dataset = test_dataset
  )

trainer.train()

In [ ]:
trainer.evaluate(test_dataset)

{'eval_loss': 0.012052150443196297}

In [ ]:
trainer.predict(test_dataset)

In [ ]:
trainer.predict(test_dataset)[1].shape

In [ ]:
output=trainer.predict(test_dataset)[1]

In [ ]:
output

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test , output)
cm

In [ ]:
trainer.save_model("spam_detector")